<a href="https://colab.research.google.com/github/harshil8991/test-repo/blob/master/optimize_CNN_using_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install keras-tuner


     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=83905b68e1b5c3829124a570d9f8a59d3eee06574de93309f6899fbe786b7d2b
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=5077348a3f8940f6b6e6004c2afe27f7a15fddc54c11bd48f4ef4048d648d0ca
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [5]:
print(tf.__version__)

2.2.0


In [6]:
fashion_mnist=keras.datasets.fashion_mnist

In [7]:
(train_images,train_labels),(test_images,test_label) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [8]:
train_images=train_images/255.0
test_images=test_images/255.0
train_images[0].shape

(28, 28)

In [9]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [12]:
tuner_search = RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)


Epoch 1/3
1688/1688 [==============================] - 16s 9ms/step - loss: 0.4217 - accuracy: 0.8447 - val_loss: 0.3605 - val_accuracy: 0.8725
Epoch 2/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2794 - accuracy: 0.8975 - val_loss: 0.2793 - val_accuracy: 0.8967
Epoch 3/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2294 - accuracy: 0.9155 - val_loss: 0.2445 - val_accuracy: 0.9088


Epoch 1/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.4012 - accuracy: 0.8551 - val_loss: 0.2982 - val_accuracy: 0.8878
Epoch 2/3
1688/1688 [==============================] - 14s 9ms/step - loss: 0.2603 - accuracy: 0.9039 - val_loss: 0.2635 - val_accuracy: 0.9022
Epoch 3/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2086 - accuracy: 0.9220 - val_loss: 0.2715 - val_accuracy: 0.9033


Epoch 1/3
1688/1688 [==============================] - 18s 11ms/step - loss: 0.4907 - accuracy: 0.8264 - val_loss: 0.3785 - val_accuracy: 0.8617
Epoch 2/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.3499 - accuracy: 0.8702 - val_loss: 0.3781 - val_accuracy: 0.8602
Epoch 3/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.3185 - accuracy: 0.8816 - val_loss: 0.3571 - val_accuracy: 0.8677


Epoch 1/3
1688/1688 [==============================] - 19s 11ms/step - loss: 0.5125 - accuracy: 0.8199 - val_loss: 0.3682 - val_accuracy: 0.8663
Epoch 2/3
1688/1688 [==============================] - 18s 11ms/step - loss: 0.3703 - accuracy: 0.8633 - val_loss: 0.3771 - val_accuracy: 0.8605
Epoch 3/3
1688/1688 [==============================] - 18s 11ms/step - loss: 0.3443 - accuracy: 0.8724 - val_loss: 0.3701 - val_accuracy: 0.8723


Epoch 1/3
1688/1688 [==============================] - 18s 11ms/step - loss: 0.4133 - accuracy: 0.8499 - val_loss: 0.3050 - val_accuracy: 0.8892
Epoch 2/3
1688/1688 [==============================] - 18s 10ms/step - loss: 0.2775 - accuracy: 0.8981 - val_loss: 0.2750 - val_accuracy: 0.8973
Epoch 3/3
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2278 - accuracy: 0.9147 - val_loss: 0.2648 - val_accuracy: 0.9018


INFO:tensorflow:Oracle triggered exit


In [14]:
model=tuner_search.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 80)        2080      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 48)        96048     
_________________________________________________________________
flatten (Flatten)            (None, 19200)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1228864   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,327,642
Trainable params: 1,327,642
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1906 - accuracy: 0.9285 - val_loss: 0.2540 - val_accuracy: 0.9120
Epoch 5/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1573 - accuracy: 0.9408 - val_loss: 0.2685 - val_accuracy: 0.9103
Epoch 6/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1267 - accuracy: 0.9530 - val_loss: 0.2739 - val_accuracy: 0.9103
Epoch 7/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1005 - accuracy: 0.9633 - val_loss: 0.3060 - val_accuracy: 0.9072
Epoch 8/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0826 - accuracy: 0.9692 - val_loss: 0.3383 - val_accuracy: 0.9140
Epoch 9/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0672 - accuracy: 0.9751 - val_loss: 0.3532 - val_accuracy: 0.9112
Epoch 10/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0529 - accuracy: 0.9802 - val_loss: 0.3928 - val_